In [2]:
#r @"..\CHSearch\bin\Debug\Newtonsoft.Json.dll"
#r @"..\CHSearch\bin\Debug\CHSearch.exe"
#r @"..\CHSearch\bin\Debug\PocoGeneration.exe"
#r @"..\CHSearch\bin\Debug\LiteDB.dll"
#r @"C:\Program Files (x86)\Reference Assemblies\Microsoft\Framework\.NETFramework\v4.6.1\System.Net.Http.dll"

In [3]:
open System
open Onnea
open Newtonsoft.Json
open Onnea.Domain
open System.IO

In [4]:
System.IO.Directory.SetCurrentDirectory(@"C:\Projects\CHSearch\CHSearch")

In [5]:
let db = Commands.GetDatabase( @"C:\temp\CHSearch\db\temp.db" )

In [6]:
let fetchedCompanyInfos = Commands.FetchCompanyInfos( db, from = 314330, count = 150 );
let fetched = fetchedCompanyInfos |> Seq.toList //|> Seq.length
let fromWeb = fetched |> Seq.where (fun r -> r.WasFetchedFromWeb) |> Seq.toList
[| fetched; fromWeb |] |> Seq.map Seq.length 

seq [150; 0]

Fetching company info from 314330 to 314479

Waiting for the fetcher to finish...
Done fetching company info from 314330 to 314479


In [7]:
let companiesByLetter = Commands.GetCompaniesWhere( db, "CompanyName",
                              fun c -> c.AsString <> null && c.AsString.StartsWith( "D" ) )
let company314128 = Commands.GetCompaniesWhere( db, "_id", // aka "CompanyInfoId"
                     fun c -> c.AsInt32 = 314128 ) |> List.ofSeq
let activeByLetter = companiesByLetter |> Seq.toList |> Seq.filter (fun c -> c.CompanyStatus = "active") |> Seq.toList
activeByLetter 
company314128

[CompanyInfo[00314128, BENN & SONS (HALIFAX) LIMITED]]

In [8]:
let allCompanies = Commands.GetCompanies( db, fun _ -> true ) |> Seq.filter (fun c -> not c.DoesNotExist) |> List.ofSeq
allCompanies |> Seq.map (fun c -> c.CompanyName)  |> Seq.take 10

seq
  ["POST & TIMES SERIES LIMITED"; "SHS INTERNATIONAL LTD";
   "NEWSQUEST (LANCS FREE) LIMITED"; "PERSHORE CLUB;LIMITED(THE)"; ...]

In [9]:
let companiesAndTheirFilingHistories = 
  allCompanies 
  |> Seq.map (fun c -> c.CompanyName, Commands.GetFilingHistory( db, c, DateTime.Parse( "2018-05-22" )))
companiesAndTheirFilingHistories

seq
  [("POST & TIMES SERIES LIMITED", FilingHistory[112005, as of: 2018-05-29]);
   ("SHS INTERNATIONAL LTD", FilingHistory[112075, as of: 2018-05-29]);
   ("NEWSQUEST (LANCS FREE) LIMITED", FilingHistory[112092, as of: 2018-05-29]);
   ("PERSHORE CLUB;LIMITED(THE)", FilingHistory[112134, as of: 2018-05-29]); ...]

In [10]:
let filingHistoriesOfCompsWithFullAccounts = 
    companiesAndTheirFilingHistories 
    |> Seq.where (fun h -> (h |> snd).Items |> Seq.exists (fun i -> i.Description.ToLower().Contains("full")))
    |> List.ofSeq
filingHistoriesOfCompsWithFullAccounts |> Seq.take 10

seq
  [("POST & TIMES SERIES LIMITED", FilingHistory[112005, as of: 2018-05-29]);
   ("SHS INTERNATIONAL LTD", FilingHistory[112075, as of: 2018-05-29]);
   ("NEWSQUEST (LANCS FREE) LIMITED", FilingHistory[112092, as of: 2018-05-29]);
   ("PERSHORE CLUB;LIMITED(THE)", FilingHistory[112134, as of: 2018-05-29]); ...]

In [17]:
let firstCompanyDocuments = 
    filingHistoriesOfCompsWithFullAccounts |> Seq.head
    |> fun nameAndFilingHistory -> (nameAndFilingHistory |> snd).Items |> Seq.filter (fun i -> i.Links <> null) 

In [21]:
firstCompanyDocuments 
|> Seq.map (fun i -> i.Category) |> Seq.toList |> List.distinct

["gazette"; "officers"; "dissolution"; "annual-return"; "accounts"]

In [29]:
let exampleAccounts = 
 firstCompanyDocuments
 |> Seq.filter (fun i -> i.Category = "accounts")
 |> Seq.map (fun i -> i.Category, Commands.GetDocument(i))
 |> Seq.head
 |> fun catAndDoc -> printfn "%s: %s"(catAndDoc |> fst) (catAndDoc |> snd).TextFile; (catAndDoc |> snd).TextFile

accounts: C:\temp\CHSearch\documents\text\https___document-api.companieshouse.gov.uk_document_0zIpHaa_x1uJzd-kMGXQI6zQEoAFIncpA0bYetk7Km8.txt


In [36]:
let turnoverLines = 
 exampleAccounts |> File.ReadAllLines
 |> Seq.filter (fun (line:string) -> line.ToLower().Contains("revenue") ) |> List.ofSeq
turnoverLines

["Advertismg revenues £ 874,000 1,] 16,000";
 "Circulation revenues £ 168,000 245,000"; "Other revenues £ 3 1,000 205,000";
 "Total revenues £ 1,073,000 1.567.000";
 "stated aﬁer the deduction of trade discounts Revenue is recognised on the publication of advertismg or on the";
 "sale of newspapers Other revenue 15 recognised as the goods or servtces are provtded Turnover con5ists"]